In [3]:
'''
以下代码将实现以下功能

1. 迁移权重模型 MobileNet-v2,并改造为单支网络
2. 定义Siamese双支网络欧式距离
3. 根据距离大小定义样本对是否匹配
4. 定义输入样本对生成器（具体代码实现参考 from utils.images_generator import generator）
5. 通过model.fit_generator训练模型，保存模型
'''

'\n以下代码将实现以下功能\n\n1. 迁移权重模型 MobileNet-v2,并改造为单支网络\n2. 定义Siamese双支网络欧式距离\n3. 根据距离大小定义样本对是否匹配\n4. 定义输入样本对生成器（具体代码实现参考 from utils.images_generator import generator）\n5. 通过model.fit_generator训练模型，保存模型\n'

In [1]:
import os
import cv2
import csv
import tensorflow as tf
import numpy as np
import random
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.optimizers import SGD,RMSprop
from keras.models import Model, Sequential,load_model
from keras.layers import Input,Concatenate, Add,Subtract,Lambda,GlobalAveragePooling2D,Dense,Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam,RMSprop,SGD

Using TensorFlow backend.


In [2]:
#set GPU to memory-growth mode

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# calculate euclidean distance
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


# calculate loss
def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

# calculate accuracy
def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    print('pred:', pred)
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [4]:
from keras.applications import ResNet50,MobileNetV2,DenseNet121
from models.base_model import base_model


weights_path='/root/tf-2.0/Siamese_Face_Detection/models/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
initial_model=MobileNetV2(weights=weights_path,include_top=False)


Inp=Input((224,224,3))
x=initial_model(Inp)
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.1)(x)
pred=Dense(50,activation='relu')(x)

base_network=Model(inputs=Inp,outputs=pred)
base_network.layers[0].trainable = False
base_network.layers[1].trainable = False

base_network.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_None (Model multiple                  2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                6450      
Total params: 2,428,402
Trainable params: 170,418
Non-trainable params: 2,257,984
___________________________________________

In [5]:
input_shape=(224,224,3)
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 50)           2428402     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]              

In [6]:
#download training images pair
train_csv_path='/root/tf-2.0/Siamese_Face_Detection/data/FaceV5_160_train.csv'
test_csv_path='/root/tf-2.0/Siamese_Face_Detection/data/FaceV5_160_test.csv'

rows_train = csv.reader(open(train_csv_path, 'r'), delimiter=',')
imgs_train = list(rows_train)

rows_test=csv.reader(open(test_csv_path, 'r'), delimiter=',')
imgs_test = list(rows_test)

#define iterations
epochs = 5
batch_size = 20
train_iterations = len(imgs_train)//20
test_iterations = len(imgs_test)//20

#choose the proper optimizer
initial_lr=0.001
decay_rate=initial_lr/epochs
sgd=SGD(learning_rate=initial_lr,decay=decay_rate,momentum=0.9)
rms=RMSprop()
adam = Adam(0.0001)

In [7]:
from utils.images_generator import generator

model_result = '/root/tf-2.0/Siamese_Face_Detection/h5/'
#define training iterator
train_data=generator(imgs_train, batch_size,input_shape)
valid_data=validation_data = generator(imgs_test,batch_size,input_shape)

checkpoint = ModelCheckpoint(filepath=model_result + 'Siamese_mobilenetv2_{epoch:03d}.h5', verbose=1)
model.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])
history=model.fit_generator(train_data,
                            steps_per_epoch = train_iterations,
                            epochs = epochs,
                            validation_data = valid_data,
                            validation_steps = test_iterations,
                            callbacks=[checkpoint])

Epoch 1/5
3000/3000 [==============================] - 224s 75ms/step - loss: 0.0979 - accuracy: 0.9231 - val_loss: 0.0839 - val_accuracy: 0.9318

Epoch 00001: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_mobilenetv2_001.h5
Epoch 2/5
3000/3000 [==============================] - 217s 72ms/step - loss: 0.0812 - accuracy: 0.9250 - val_loss: 0.0518 - val_accuracy: 0.9169

Epoch 00002: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_mobilenetv2_002.h5
Epoch 3/5
3000/3000 [==============================] - 217s 72ms/step - loss: 0.0763 - accuracy: 0.9236 - val_loss: 0.0800 - val_accuracy: 0.9110

Epoch 00003: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_mobilenetv2_003.h5
Epoch 4/5
3000/3000 [==============================] - 216s 72ms/step - loss: 0.0718 - accuracy: 0.9242 - val_loss: 0.1118 - val_accuracy: 0.8740

Epoch 00004: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_mobilenetv2_004.h5
Epoch 5/5
3000/3000 [===========